In [8]:
import pandas as pd

# Carregamento dos dados
caminho_arquivo = 'animes.csv'
dados_animes = pd.read_csv(caminho_arquivo)

# Renomeando e preparando colunas
dados_animes.rename(columns={
    'rate': 'Avaliação do anime',
    'votes': 'Votos totais do anime',
    'episodes': 'Quantidade de episódios do anime'
}, inplace=True)

# Exclusão de animes sem episódios
dados_animes = dados_animes[dados_animes['Quantidade de episódios do anime'] > 0]

# Contagem total de votos
dados_animes['Contagem total de votos de todos os animes'] = dados_animes['Votos totais do anime'].sum()

# Preparação e cálculo das estatísticas por gênero
colunas_genero = [col for col in dados_animes.columns if 'genre_' in col]
estatisticas_genero = pd.DataFrame()

for genero in colunas_genero:
    genero_label = genero.replace('genre_', '').capitalize()
    grupo = dados_animes[dados_animes[genero] == 1]

    # Calcular estatísticas e armazenar em um DataFrame temporário
    resumo_genero = pd.DataFrame({
        f'Contagem de votos do gênero {genero_label}': [grupo['Votos totais do anime'].sum()],
        f'Contagem de animes do gênero {genero_label}': [grupo.shape[0]],
        f'Contagem de episódios do gênero {genero_label}': [grupo['Quantidade de episódios do anime'].sum()],
        f'Média de avaliações do gênero {genero_label}': [grupo['Avaliação do anime'].mean()],
        f'Assimetria da média do gênero {genero_label}': [grupo['Avaliação do anime'].skew()],
        f'Curtose da média do gênero {genero_label}': [grupo['Avaliação do anime'].kurt()],
        f'Desvio padrão da média do gênero {genero_label}': [grupo['Avaliação do anime'].std()],
        'Gênero Calculado': genero_label
    })
    estatisticas_genero = pd.concat([estatisticas_genero, resumo_genero], ignore_index=True)

# Classificação de avaliação
def classificar_avaliacao(valor):
    if valor <= 1:
        return "Péssimo"
    elif valor <= 2:
        return "Ruim"
    elif valor <= 3:
        return "Neutro"
    elif valor <= 4:
        return "Bom"
    else:
        return "Excelente"

dados_animes['Avaliação Categoria do anime'] = dados_animes['Avaliação do anime'].apply(classificar_avaliacao)

# Adicionar as estatísticas de gênero ao DataFrame principal (pode ajustar conforme necessidade de apresentação)
dados_animes = pd.concat([dados_animes, estatisticas_genero], axis=1)

# Organizando as colunas para exportação (ajuste conforme a ordem desejada)
colunas_exportacao = [
    'anime', 'Gênero do anime', 'anime_url', 'anime_img', 'Quantidade de episódios do anime',
    'Votos totais do anime', 'Avaliação do anime', 'Avaliação Categoria do anime'
] + estatisticas_genero.columns.tolist()

dados_animes = dados_animes[colunas_exportacao]

# Salvando o DataFrame processado
caminho_arquivo_processado = 'animes_TESTE.csv'
dados_animes.to_csv(caminho_arquivo_processado, index=False)

print(f"Arquivo processado salvo em: {caminho_arquivo_processado}")


C:\Users\Pref Sales Oliveira\AppData\Local\Temp\ipykernel_1716\1479935234.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dados_animes[f'Assimetria da média do gênero {genero_label}'] = grupo['Avaliação do anime'].skew()
C:\Users\Pref Sales Oliveira\AppData\Local\Temp\ipykernel_1716\1479935234.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dados_animes[f'Curtose da média do gênero {genero_label}'] = grupo['Avaliação do anime'].kurt()
C:\Users\Pref Sales Oliveira\AppData\Local\Temp\ipykernel_1716\1479935234.py:37: Perf

KeyError: "['Contagem de votos do Action', 'Contagem de animes do Action', 'Contagem de episódios do Action', 'Média de avaliações do Action', 'Assimetria da média do Action', 'Curtose da média do Action', 'Desvio padrão da média do Action', 'Contagem de votos do Adventure', 'Contagem de animes do Adventure', 'Contagem de episódios do Adventure', 'Média de avaliações do Adventure', 'Assimetria da média do Adventure', 'Curtose da média do Adventure', 'Desvio padrão da média do Adventure', 'Contagem de votos do Comedy', 'Contagem de animes do Comedy', 'Contagem de episódios do Comedy', 'Média de avaliações do Comedy', 'Assimetria da média do Comedy', 'Curtose da média do Comedy', 'Desvio padrão da média do Comedy', 'Contagem de votos do Drama', 'Contagem de animes do Drama', 'Contagem de episódios do Drama', 'Média de avaliações do Drama', 'Assimetria da média do Drama', 'Curtose da média do Drama', 'Desvio padrão da média do Drama', 'Contagem de votos do Family', 'Contagem de animes do 